![](img/cabecera.png)


# **Taller AWS: API REST en AWS EC2**

Para este taller vas a desplegar una API REST desarrollada con Flask en la nube de Amazon Web Services (AWS), de manera que puedas ejecutar tu aplicación web de forma remota y acceder a ella desde cualquier lugar.

En el taller seguiremos los siguientes pasos:
- Crearemos una máquina virtual en una instancia de AWS
- Instalaremos Python, Flask y todas las dependencias necesarias. 
- Configuraremos el acceso y desplegaremos nuestra API REST usando Gunicorn como servidor de aplicaciones.
- Accederás a tu API desde cualquier navegador usando la dirección IP pública de tu instancia.

## Crea una cuenta de AWS

Lo primero que necesitas es una cuenta de AWS. Puedes crear la cuenta desde este [enlace](https://aws.amazon.com/free/), además de poder consultar los servicios gratuitos del Free Tier.

## Recurso EC2

Una vez hayas accedido a tu cuenta, tienes que crear un recurso EC2, las instancias EC2 son máquinas virtuales en la nube de AWS. Todos los proveedores Cloud disponen de servicios similares o iguales, pero sus nombres específicos cambian por cuestiones de marketing.  

En el buscador (arriba a la izquierda) escribe EC2 o pulsa en el menú de Servicios (el icono con cuadraditos) y ve a Informática -> EC2. Una vez estemos en esta sección, seleccionamos "Lanzar una instancia".

<img src="img/imagen-01.png" alt="AWS Dashboard" style="width: 800px;">  


## Configuración del EC2

Ahora tendremos que escoger una Amazon Machine Image (AMI), donde configuraremos el sistema operativo, recursos y software que vayamos a utilizar.

Para este taller vamos a utilizar **Ubuntu Server 24.04 LTS** que está incluida en el Free Tier.  

Ubuntu es una distribución de Linux muy popular y viene con herramientas que nos facilitarán la instalación de Python y nuestras dependencias.

En "Imágenes de aplicaciones y sistemas operativos (Imagen de máquina de Amazon)" busca y selecciona:
- **Ubuntu Server 24.04 LTS (HVM), SSD Volume Type**

Para la capacidad de la instancia, utilizaremos una instancia de tipo **t3.micro** compatible con la Capa Gratuita o Free Tier, que nos proporciona:
- 2 vCPU
- 1 GiB de RAM
- 8 GiB de Almacenamiento SSD EBS

<img src="img/imagen-02.png" alt="AWS EC2 Ubuntu Configuration" style="width: 800px;">  


## Claves de la máquina

Para poder acceder a la máquina sin que nos pida las credenciales, necesitaremos una clave privada, que conservaremos en local y una clave pública que estará en el EC2. Esto se denomina "key-pair".

**IMPORTANTE.** Descárgate la clave privada (archivo .pem). Es el único momento en el que podrás hacerlo. Para este taller la llamaremos "flask-api-key", por lo que descargaremos un archivo "flask-api-key.pem".

¡Ya tienes la máquina desplegada y corriendo! En los siguientes apartados aprenderemos a acceder a la misma, instalar Python, Flask y desplegar nuestra API.

<img src="img/imagen-03.png" alt="Key Pair Flask Configuration 01" style="width: 800px;">  


<img src="img/imagen-04.png" alt="Key Pair Flask Configuration 02" style="width: 800px;">  


## Configuración del Security Group

En el desplegable de "Configuraciones de Red" debemos buscar la parte referente a "Firewall (Grupos de Seguridad)". Para este taller vamos a crear un nuevo grupo de seguridad con una configuración inicial permisiva, este grupo de seguridad será específico para nuestra API REST por lo que debemos seleccionar:

- **Crear grupo de seguridad**, el nombre será asignado automáticamente.
- **Permitir el tráfico de SSH desde**
- **Permitir el tráfico de HTTPS desde Internet**
- **Permitir el tráfico de HTTP desde Internet**

**Nota de seguridad**: Acabas de crear un grupo de seguridad que **permita todo el tráfico** para facilitar las pruebas iniciales. Esta configuración es solo para desarrollo. En producción nunca permitas todo el tráfico.

<img src="img/imagen-05.png" alt="Security Group Flask Configuration" style="width: 800px;">  


# Acceso al EC2 desde local

Ya tenemos la máquina lista. Ahora lo que queda es acceder a la misma desde nuestro local. Para ello necesitamos conectarnos por SSH al EC2. Si pulsas el botón "Conectar" en el panel de la instancia verás diferentes formas de conectarte a tu instancia.  

<img src="img/imagen-06.png" alt="Conect to Instance" style="width: 800px;">  

**Para Ubuntu**, el usuario por defecto es `ubuntu` (en cada SO el usuario por defecto es diferente, por ejemplo en máquinas con Amazon Linux el usuario por defecto es `ec2-user`).

Si estás en Windows, abre un terminal de GitBash en el directorio donde hayas descargado tu archivo .pem y ejecuta las siguientes sentencias o pasos. Si estás en MacOS/Linux, simplemente abre un terminal y sigue los pasos.

Para evitar problemas de permisos con la clave, ejecuta:

```bash
chmod 400 flask-api-key.pem
```
Una vez cambiados los permisos podemos conectarnos a la instancia ejecutando la sentencia:

```bash
ssh -i "flask-api-key.pem" ubuntu@ec2-XXX-XXX-XXX-XXX.compute-1.amazonaws.com
```

**Recuerda:** Sustituye la dirección DNS por la de tu instancia, que puedes encontrar en el panel de EC2.


<img src="img/imagen-07.png" alt="EC2 SSH Connection" style="width: 600px;">


# Preparación de la instancia EC2

Una vez conectados a nuestra instancia Ubuntu, vamos a preparar el entorno para nuestra API Flask.

## Actualización de dependencias

Lo primero es actualizar el sistema con los últimos paquetes:

```bash
sudo apt update -y
```

Este comando:
- `sudo`: ejecuta como administrador
- `apt update`: actualiza la lista de paquetes disponibles
- `-y`: acepta automáticamente todas las confirmaciones

## Instalación de herramientas esenciales

Nuestro servidor de Ubuntu ya tiene Python 3 instalado, pero carece de algunas características que nos son muy familiares, como por ejemplo pip (gestor de paquetes de Python) y git (para clonar repositorios). Si además quieres actualizar Python a la última distribución disponible para Ubuntu, puedes incluirlo en la sentencia:

```bash
sudo apt install python3 python3-pip git -y
```

Verificamos que todo se instaló correctamente:

```bash
python3 --version
pip3 --version
git --version
```

## Clonado del proyecto

Ahora que tenemos Git instalado, clonamos el repositorio de nuestro proyecto Flask y comprobamos que se haya clonado correctamente:

```bash
git clone https://github.com/tu-usuario/tu-proyecto-flask.git
ls
```

**Nota:** Sustituye la URL por la de tu repositorio real.

# Instalación de dependencias del proyecto

Entramos en el directorio del proyecto e intentamos instalar las dependencias:

```bash
cd tu-proyecto-flask
pip install -r requirements.txt
```

¡Vaya!, es probable que encuentres un error (bastante verboso) que diga algo similar a:

```
error: externally-managed-environment
```


## ¿Por qué Ubuntu no nos deja instalar dependencias?

Ubuntu 22.04 y versiones posteriores incluyen una protección que previene la instalación de paquetes Python directamente en el sistema con pip. Python es parte esencial del funcionamiento de la máquina Ubuntu y por este motivo se impide la instalación de paquetes sobre el entorno del sistema. De esta forma conseguimos:

1. **Evitar conflictos** entre paquetes del sistema y paquetes instalados por el usuario
2. **Proteger el sistema** de posibles incompatibilidades
3. **Fomentar buenas prácticas** como el uso de entornos virtuales

**La solución:** Usar entornos virtuales (Virtual Environments) para no romper nuestra máquina.

## Creación de entorno virtual

Al igual que ocurría con pip y Git, antes de poder crear entornos virtuales debemos instalar el paquete para ello:

```bash
sudo apt install python3-venv -y
```

Ahora creamos nuestro entorno virtual, **pero es muy IMPORTANTE que lo hagamos dentro de la carpeta de nuestro proyecto**:

```bash
python3 -m venv venv
```

Activamos el entorno virtual:

```bash
source venv/bin/activate
```

Verás que tu prompt cambia y muestra `(venv)` al principio, indicando que el entorno está activo.

Ahora sí podemos instalar las dependencias:

```bash
pip install -r requirements.txt
```

# Lanzamiento de la API

## ¿Qué es Gunicorn?

Ahora que ya estamos familiarizados con Gunicorn (Green Unicorn), recuerda que es un servidor WSGI HTTP para Python. Es mucho más robusto y eficiente que el servidor de desarrollo integrado de Flask para entornos de producción. Entre sus principales ventajas podemos destacar:

- Manejo de múltiples workers (procesos)
- Mejor rendimiento bajo carga
- Más estable para producción
- Configuración flexible

## Instalación de Gunicorn

Nuestra recomendación es que Gunicorn se encuentre en tu requirements.txt, de modo que al instalar todas las dependencias se incluya la instalación de Gunicorn, en caso contrario deberás instalarlo con pip.

## Lanzamiento de la API

Lanzamos nuestra API Flask usando Gunicorn con 3 workers y *binded* o vinculado a todas las interfaces de red en el puerto 5000:

```bash
gunicorn -w 3 -b 0.0.0.0:5000 app:app
```

**Explicación del comando:**
- `gunicorn`: el servidor WSGI
- `-w 3`: el servidor tendrá 3 *workers* para gestionar la carga
- `-b 0.0.0.0:5000`: *bind* (vincular) a todas las interfaces de red en el puerto 5000
- `app:app`: el primer `app` es el archivo Python (app.py), el segundo `app` es la instancia de Flask dentro del archivo

**¿Por qué `0.0.0.0` y no `localhost`?**  

- `localhost (127.0.0.1)`: Solo acepta conexiones desde la propia máquina
- `0.0.0.0`: Acepta conexiones desde cualquier IP (necesario para acceso desde internet)

Como queremos acceder desde nuestro navegador, necesitamos `0.0.0.0`.

**¿Por qué 3 workers?**  

Regla: `(CPU cores × 2) + 1`. Para t3.micro (2 vCPU) podríamos usar un máximo de 5 workers, pero con solo 1GiB RAM el equilibrio entre rendimiento y estabilidad está en 3 workers.

Si todo va bien, verás algo parecido a:
```
[INFO] Starting gunicorn 23.0.0
[INFO] Listening at: http://0.0.0.0:5000
[INFO] Using worker: sync
[INFO] Booting worker with pid: XXX1
[INFO] Booting worker with pid: XXX2
[INFO] Booting worker with pid: XXX3
```

# Acceso a la API

## Obtención de la IP pública

Para acceder a tu API desde cualquier lugar, necesitas la **IP pública** de tu instancia EC2:

1. Ve al panel de EC2 en AWS
2. Selecciona tu instancia
3. En la pestaña "Detalles", busca **"Dirección IPv4 pública"**
4. Copia esta dirección (será algo como: `54.123.45.67`)

<img src="img/imagen-08.png" alt="API Access Information" style="width: 800px;">


## Conexión a la API

Ahora puedes acceder a tu API desde cualquier navegador usando:

```
http://TU_IP_PUBLICA:5000
```

Por ejemplo:
```
http://54.123.45.67:5000
```


**Recuerda que esto no solo lo puedes probar desde tu navegador, también puedes usar herramientas como:**
- **Postman** para probar endpoints
- **curl** desde la línea de comandos del terminal

# ¿Ha funcionado?

En caso afirmativo, ¡es una sorpresa interesante!  

En caso negativo, ¿qué ha podido fallar?

Lo que ocurre es que Gunicorn se está ejecutando en el puerto 5000 del Localhost de la instancia EC2, que es diferente al Localhost de nuestro ordenador personal.

### Opción 1: Crear un túnel SSH entre nuestro equipo y la instancia EC2

Abre un nuevo GitBash o terminal y ejecuta la siguiente sentencia para establecer un túnel entre tu equipo y la instancia remota.

```bash
ssh -i "flask-api-key.pem" -L localhost:5000:localhost:5000 ubuntu@ec2-XXX-XXX-XXX-XXX.compute-1.amazonaws.com
```

Con el túnel levantado, prueba a acceder en http://127.0.0.1:5000/ con tu navegador.

Esta opción es segura pero solo te permite acceder a tu API desde tu ordenador (no está expuesta al tráfico desde internet).

**Recuerda:** Sustituye la dirección DNS por la de tu instancia, que puedes encontrar en el panel de EC2.


### Opción 2: Refinamiento del Grupo de Seguridad (Security Group)

Con nuestra instancia funcionando, vamos a **actualizar el grupo de seguridad** para ser más específicos:

1. En el panel de la Instancia EC2 desciende hasta que veas la pestaña Seguridad.
2. Pincha en el grupo de seguridad de la instancia.
3. **Edita las reglas de entrada** y añade:
   - **Custom TCP**: Puerto **5000**, origen 0.0.0.0/0 (aquí correrá Gunicorn).
4. **Guarda las reglas de entrada.**

**¿Por qué puerto 5000?** Gunicorn (nuestro servidor de aplicaciones) estará escuchando en este puerto.

Con las reglas modificadas prueba a acceder desde tu navegador usando la IP pública de tu instancia, recuerda incluir el puerto 5000 para confirmar que Gunicorn funciona y la API está levantada.

Nota: Una vez confirmes que la API funciona, lo más seguro es **eliminar** la regla que acabamos de crear.


*Paso 1:*  

<img src="img/imagen-09.png" alt="Security Group Flask Configuration" style="width: 800px;">


*Paso 2:*  

<img src="img/imagen-10.png" alt="Security Group Flask Configuration" style="width: 800px;">


*Paso 3:*  

<img src="img/imagen-11.png" alt="Security Group Flask Configuration" style="width: 800px;">


# ¡Enhorabuena!

Has logrado desplegar exitosamente tu API REST con Flask en AWS EC2. Tu aplicación está ahora:

- **Corriendo en la nube** - Accesible desde cualquier lugar del mundo  
- **Usando Gunicorn** - Un servidor de producción robusto  
- **En un entorno virtual** - Aislado y con dependencias controladas  

## Lo que has aprendido:

1. **Configuración de EC2** con Ubuntu Server
2. **Gestión de Security Groups** y puertos
3. **Resolución de problemas** con entornos externally-managed
4. **Uso de entornos virtuales** en Linux
5. **Despliegue con Gunicorn** en lugar del servidor de desarrollo
6. **Acceso público** a tu aplicación web

## Pero esto no acaba aquí...

Si tienes tiempo y energía, **¡continúa con la Parte Extra Opcional!**

## **Y RECUERDA**

#### **¡Apaga tu instancia cuando no la uses para conservar tu Capa Gratuita o Free Tier!**


---

## Comandos de referencia rápida

### Gestión de la instancia:
```bash
# Conectar por SSH
ssh -i "flask-api-key.pem" ubuntu@ec2-XXX-XXX-XXX-XXX.compute-1.amazonaws.com

# Activar entorno virtual
source venv/bin/activate

# Lanzar API
gunicorn -w 3 -b 0.0.0.0:5000 app:app

```

### Pruebas de la API:
```bash
# Desde tu ordenador local
curl http://TU_IP_PUBLICA:5000
curl http://TU_IP_PUBLICA:5000/api/v1/predict?tv=50&radio=75.5&newspaper=200
```